# MAP Score using Word2Vec
1.   with Rocchio Algorithm, without Query expansion- 0.303
2.   with Rocchio Algorithm and with Query expansion- 0.210


# Extracting the Data

In [ ]:
!gdown --id 1JuawXQmYVkjpfL3H0blqjDrqw8V1lHrC

Downloading...
From: https://drive.google.com/uc?id=1JuawXQmYVkjpfL3H0blqjDrqw8V1lHrC
To: /home/akarofl/Desktop/Projects/Makkhi/IR A6/FIRE_Dataset_EN_2010.rar
133MB [02:04, 1.07MB/s]


In [ ]:
!unrar x FIRE_Dataset_EN_2010.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from FIRE_Dataset_EN_2010.rar

Creating    FIRE_Dataset_EN_2010                                      OK
Extracting  FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt.gz             0  OK 
Extracting  FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt               0  OK 
Extracting  FIRE_Dataset_EN_2010/English-Data.tgz                             1 1 1 2 2 2 3 3 3 4 4 4 5 5 5 6 6 6 6 7 7 7 8 8 8 9 9 9 99  OK 
All OK


In [ ]:
!tar -xvzf FIRE_Dataset_EN_2010/English-Data.tgz

ry_7925796.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070722_sports_story_8091125.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070416_sports_story_7652990.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070310_sports_story_7497024.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070702_sports_story_8003930.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070523_sports_story_7817201.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070421_sports_story_7676644.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070325_sports_story_7562923.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070702_sports_story_8003598.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070227_sports_story_7445769.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070128_sports_story_7316676.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070222_sports_story_7424674.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070727_sports_story_8112983.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070320_sports_story_7541322.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070119_sports_story_7281837.utf8
TELEGRAPH_UTF8/2007_utf8/sports/1070831_sports_story_8258236

In [ ]:
!gzip -d FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt.gz

# Importing and downloading the required packages

In [ ]:
import os
import glob
from bs4 import BeautifulSoup
import lxml.html
import re
import numpy as np    
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
import bisect
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from gensim.models import Word2Vec

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/akarofl/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/akarofl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/akarofl/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/akarofl/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Extracting the data of document files using BeautifulSoup
In this section, data stored in document files is extracted using beautiful soup and various text preprocessing steps are performed and then the data is stored in a proper format with docname as the first element and list of words of doc as the second element in each row.

In [ ]:
files = [] # List is storing file name and file content. File content is a list of words.
corpus = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer() 
postag = nltk.corpus.wordnet
stop_words = set(stopwords.words('english'))

# Returns Part of Speech for lemmatization
def wordnet_pos(word):
    t = nltk.pos_tag([word])[0][1][0].lower()
    t_pos = {"j": postag.ADJ, "n": postag.NOUN, "v": postag.VERB, "r": postag.ADV}
    return t_pos.get(t, postag.NOUN)
i = 0
for subdir, dirs, filenames in os.walk('TELEGRAPH_UTF8/'):
    print('Progress: ',i)
    for file_obj in filenames:
      i = i+1
      filename = os.path.join(subdir, file_obj)
      # Extracting data and saving it to raw_text
      soup = BeautifulSoup(open(filename, "r").read())
      raw_text = soup.find('text').get_text()
      for sym in raw_text : 
        # Removing punctuation
        if sym in punc : 
          raw_text = raw_text.replace(sym, "")

      # Removing non-alphabetic text and Converting text to lower-case
      words = [word.lower() for word in raw_text.split() if word.isalpha()]
      # Removing stopwords
      words = [w for w in words if not w in stop_words]
      # Performing lemmatization
      lemmatized = [lemmatizer.lemmatize(word) for word in words]
      temp_files = []
      temp_files.append(filename)
      temp_files.append(lemmatized)
      # Storing file data to list
      files.append(temp_files)
      corpus.append(lemmatized)
      # Appending data to 'files' list
print("Total Documents: ", len(files))

Progress:  0
Progress:  0
Progress:  0
Progress:  675
Progress:  8290
Progress:  10565
Progress:  12283
Progress:  15911
Progress:  18148
Progress:  20103
Progress:  24403
Progress:  30447
Progress:  30447
Progress:  30480
Progress:  33434
Progress:  33462
Progress:  33513
Progress:  34176
Progress:  35090
Progress:  35174
Progress:  37289
Progress:  37415
Progress:  38579
Progress:  39515
Progress:  39534
Progress:  39585
Progress:  41800
Progress:  41857
Progress:  44791
Progress:  44791
Progress:  53896
Progress:  55889
Progress:  58543
Progress:  64793
Progress:  68121
Progress:  70923
Progress:  77355
Progress:  85890
Progress:  85890
Progress:  95433
Progress:  97786
Progress:  99917
Progress:  105419
Progress:  108496
Progress:  111250
Progress:  117397
Total Documents:  125586


# Model Training with CBOW for Word2vec

In [ ]:
vect_size = 150

print("Training with CBOW started...")
# Training Word2Vec model using CBOW 
cbow_model = Word2Vec(corpus, size = vect_size, min_count = 2,window = 10, sg = 0, hs = 1, iter = 5, workers = 10)

print("Training with CBOW done.")

Training with CBOW started...
Training with CBOW done.


# Document Vectors
In this section, document vectors are created in which vector for each word is added in the row and then it is normalised with the length of that document.

In [ ]:
print("Creating vectors for Documents using CBOW word2vec ...")
# Calculating weighted average and forming document vectors
count = 0
doc_arr = []
for i in corpus:
    demo = np.zeros((vect_size,),dtype = float) # initializing a zero matrix
    for j in i:
        try:
            demo = demo + cbow_model.wv.__getitem__(j) # adding values of each token in doc to vector
        except:
          continue
    count = count + 1
    doc_arr.append(demo/len(i))
    if count%1000 == 0:
      print("Progress:", count, "/", len(corpus))

print("Vectors created.")

Creating vectors for Documents using CBOW word2vec ...
Progress: 1000 / 125586
Progress: 2000 / 125586
Progress: 3000 / 125586
Progress: 4000 / 125586
Progress: 5000 / 125586
Progress: 6000 / 125586
Progress: 7000 / 125586
Progress: 8000 / 125586
Progress: 9000 / 125586
Progress: 10000 / 125586
Progress: 11000 / 125586
Progress: 12000 / 125586
Progress: 13000 / 125586
Progress: 14000 / 125586
Progress: 15000 / 125586
Progress: 16000 / 125586
Progress: 17000 / 125586
Progress: 18000 / 125586
Progress: 19000 / 125586
Progress: 20000 / 125586
Progress: 21000 / 125586
Progress: 22000 / 125586
Progress: 23000 / 125586
Progress: 24000 / 125586
Progress: 25000 / 125586
Progress: 26000 / 125586
Progress: 27000 / 125586
Progress: 28000 / 125586
Progress: 29000 / 125586
Progress: 30000 / 125586
Progress: 31000 / 125586
Progress: 32000 / 125586
Progress: 33000 / 125586
Progress: 34000 / 125586
Progress: 35000 / 125586
Progress: 36000 / 125586
Progress: 37000 / 125586
Progress: 38000 / 125586
Prog

# Queries are extracted
In this section, original queries are extracted and text preprocessing is performed on those queries. Then the data is stored in required format where 0th column of every row is query name and 1st column of every row is a list of words in that query.

In [ ]:
queries = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
porter = PorterStemmer()
lemmatizer = WordNetLemmatizer() 
postag = nltk.corpus.wordnet
stop_words = set(stopwords.words('english'))

import xml.etree.ElementTree as ET
queryPath = "FIRE_Dataset_EN_2010/en.topics.76-125.2010.txt"
tree = ET.parse(queryPath)
root = tree.getroot()

#Collecting Queries
for records in root.findall('top'):
    summary = records.find('num').text
    full = records.find('title').text

    words = [word.lower() for word in full.split() if word.isalpha()]
    # Removing stopwords
    words = [w for w in words if not w in stop_words]
    # Performing lemmatization
    lemmatized = [lemmatizer.lemmatize(word) for word in words]

    queries.append([summary, lemmatized])

print("Queries Collected")

Queries Collected


# Query vectors created using original queries

In [ ]:
q_arr = []
count = 0

print("Creating vectors for queries using CBOW word2vec ...")

for i in queries :
  demo = np.zeros((vect_size,),dtype=float)
  for j in i[1] :
    try:
      demo = demo + cbow_model.wv.__getitem__(j)
    except:
      continue
  count = count + 1
  print("Progress:", count, "/", len(queries))
  q_arr.append(demo/len(i[1]))

print("Vectors created.")

Creating vectors for queries using CBOW word2vec ...
Progress: 1 / 50
Progress: 2 / 50
Progress: 3 / 50
Progress: 4 / 50
Progress: 5 / 50
Progress: 6 / 50
Progress: 7 / 50
Progress: 8 / 50
Progress: 9 / 50
Progress: 10 / 50
Progress: 11 / 50
Progress: 12 / 50
Progress: 13 / 50
Progress: 14 / 50
Progress: 15 / 50
Progress: 16 / 50
Progress: 17 / 50
Progress: 18 / 50
Progress: 19 / 50
Progress: 20 / 50
Progress: 21 / 50
Progress: 22 / 50
Progress: 23 / 50
Progress: 24 / 50
Progress: 25 / 50
Progress: 26 / 50
Progress: 27 / 50
Progress: 28 / 50
Progress: 29 / 50
Progress: 30 / 50
Progress: 31 / 50
Progress: 32 / 50
Progress: 33 / 50
Progress: 34 / 50
Progress: 35 / 50
Progress: 36 / 50
Progress: 37 / 50
Progress: 38 / 50
Progress: 39 / 50
Progress: 40 / 50
Progress: 41 / 50
Progress: 42 / 50
Progress: 43 / 50
Progress: 44 / 50
Progress: 45 / 50
Progress: 46 / 50
Progress: 47 / 50
Progress: 48 / 50
Progress: 49 / 50
Progress: 50 / 50
Vectors created.


# Extracting data from Qrels
In this section, data from qrels file is extracted and stored in proper format with each value in each column of every row.

In [ ]:
file1 = open('FIRE_Dataset_EN_2010/en.qrels.76-125.2010.txt', 'r')
Lines = file1.readlines()

refDoc = []
for line in Lines:
    refDoc.append(line.split())

# Saving document names separately
In this section, we save document names from original corpus named 'files'. This is needed for the relevance feedback where we need to extract particular document vectors for each query with which its relevance is either 0 or 1.

In [ ]:
doc_names = []
for name in files : 
  doc_names.append(name[0].split('/')[3])

# Updating the query vector as per Rocchio Algorithm
In this section, we update the query vector for each query with each document using the Rocchio Algorithm. This algorithm uses the original query vector and already known information about relevance of query with each document which can be either 0 or 1.

In [ ]:
new_quer = []
a = 1
b = 0.75
c = 0.15

print("Creating new query vector using Rocchio Algorithm...")
for i in range(len(queries)) : 
  count1 = 0
  count2 = 0
  demo1 = np.zeros((vect_size,),dtype=float)
  demo2 = np.zeros((vect_size,),dtype=float)
  queryNo = queries[i][0]

  for k in range(len(refDoc)) :
    if(refDoc[k][0] == queryNo) : 
      index = doc_names.index(refDoc[k][2])
      if refDoc[k][3] == 1 : 
        demo1 = demo1 + doc_arr[index]
        count1 += 1
      if refDoc[k][3] == 0 : 
        count2 += 1
        demo2 = demo2 + doc_arr[index]

    if count1 == 0:
      count1 = 1
    if count2 == 0:
      count2 = 1
  
  temp_vect = np.zeros((vect_size,),dtype=float)
  temp_vect = temp_vect + np.multiply(q_arr[i], a) + np.multiply(demo1, b/count1) - np.multiply(demo2, c/count2)
  new_quer.append(temp_vect)

q_arr = new_quer
del new_quer
print("New Query vector created using Rocchio Algorithm.")

Creating new query vector using Rocchio Algorithm...
New Query vector created using Rocchio Algorithm.


# Calculating the cosine similarity
In this section, the cosine similarity is calculated between the new query vector and the original document vector. And then rankings are calculated by sorting the similarity score.

In [ ]:
from numpy import dot
from numpy.linalg import norm

print("Calculating Rankings of Docs for each query...")

r1 = []

for i in range(len(q_arr)) :
    rTemp = []
    for j in range(len(doc_arr)) :
        cos_sim = dot(q_arr[i], doc_arr[j])/(norm(q_arr[i])*norm(doc_arr[j]))
        rTemp.append([files[j][0], cos_sim])
        
    rTemp.sort(key = lambda rTemp: rTemp[1], reverse=True) 
    r1.append(rTemp)

    print("Progress: ", i+1, " / ", len(q_arr))

print("Ranking Complete.")

Calculating Rankings of Docs for each query...
Progress:  1  /  50
Progress:  2  /  50
Progress:  3  /  50
Progress:  4  /  50
Progress:  5  /  50
Progress:  6  /  50
Progress:  7  /  50
Progress:  8  /  50
Progress:  9  /  50
Progress:  10  /  50
Progress:  11  /  50
Progress:  12  /  50
Progress:  13  /  50
Progress:  14  /  50
Progress:  15  /  50
Progress:  16  /  50
Progress:  17  /  50
Progress:  18  /  50
Progress:  19  /  50
Progress:  20  /  50
Progress:  21  /  50
Progress:  22  /  50
Progress:  23  /  50
Progress:  24  /  50
Progress:  25  /  50
Progress:  26  /  50
Progress:  27  /  50
Progress:  28  /  50
Progress:  29  /  50
Progress:  30  /  50
Progress:  31  /  50
Progress:  32  /  50
Progress:  33  /  50
Progress:  34  /  50
Progress:  35  /  50
Progress:  36  /  50
Progress:  37  /  50
Progress:  38  /  50
Progress:  39  /  50
Progress:  40  /  50
Progress:  41  /  50
Progress:  42  /  50
Progress:  43  /  50
Progress:  44  /  50
Progress:  45  /  50
Progress:  46  / 

# Calculating the MAP score using top 10 documents for each query

In [ ]:
#List containing AP10 for each query
AP10 = []

for i in range(len(r1)):
    cAP = 0
    tAP = []
    for j in range(min(10, len(r1[i]))):
        for l in range(j):
            docName = r1[i][j][0].split('/')[3]
            queryNo = queries[i][0]

            #Search for this doc in reference file
            for k in range(len(refDoc)):
                if(refDoc[k][0] == queryNo and refDoc[k][2] == docName and refDoc[k][3] == '1'):
                    cAP = cAP + 1
        tAP.append(cAP/(j+1))
    avgP = 0
    for val in tAP:
        avgP += val
    AP10.append(avgP/10)

MAP10 = 0
for ap in AP10:
    MAP10 += ap
MAP10 /= 50

In [ ]:
print('MAP using CBOW for new query vector using Rocchio Algorithm :', round(MAP10,3))

MAP using CBOW for new query vector using Rocchio Algorithm : 0.303


# Query Expansion using Nltk Wordnet
In this section, we append the synonyms for each word of each query using nltk wordnet. Then, text preprocessing is performed on the appended words.

In [ ]:
from nltk.corpus import wordnet as wn
exp_quer = []

for i in range(len(queries)) : 
  temp_quer = []
  # appending the original words 
  temp_quer.append(queries[i][0])
  temp_terms = []
  for word in queries[i][1] : 
    # finding synonyms using wordnet and appending to the syn_list
    syns = wn.synsets(word) 
    syn_list = []
    for j in range(len(syns)) : 
      syn_list.append(syns[j].lemmas()[0].name())
    syn_list = list(set(syn_list))
    # preprocessing on appended words
    words = [w for w in syn_list if not w in stop_words]
    words = [word.lower() for word in words if word.isalpha()]
    lemmatized = [lemmatizer.lemmatize(word) for word in words]

    # taking at most 10 synonyms for each word
    for k in range(min(10, len(lemmatized))):
      temp_terms.append(lemmatized[k])
    temp_terms.append(word)

  temp_quer.append(list(set(temp_terms)))
  exp_quer.append(temp_quer)

# Query vectors for expanded queries
In this section, we generate query vectors for expanded queries the same way we did for the original queries

In [ ]:
q_arr = []
count = 0

print("Creating vectors for queries using CBOW word2vec ...")

for i in exp_quer :
  demo = np.zeros((vect_size,),dtype=float)
  for j in i[1] :
    try:
      demo = demo + cbow_model.wv.__getitem__(j)
    except:
      continue
  count = count + 1
  print("Progress:", count, "/", len(exp_quer))
  q_arr.append(demo/len(i[1]))

print("Vectors created.")

Creating vectors for queries using CBOW word2vec ...
Progress: 1 / 50
Progress: 2 / 50
Progress: 3 / 50
Progress: 4 / 50
Progress: 5 / 50
Progress: 6 / 50
Progress: 7 / 50
Progress: 8 / 50
Progress: 9 / 50
Progress: 10 / 50
Progress: 11 / 50
Progress: 12 / 50
Progress: 13 / 50
Progress: 14 / 50
Progress: 15 / 50
Progress: 16 / 50
Progress: 17 / 50
Progress: 18 / 50
Progress: 19 / 50
Progress: 20 / 50
Progress: 21 / 50
Progress: 22 / 50
Progress: 23 / 50
Progress: 24 / 50
Progress: 25 / 50
Progress: 26 / 50
Progress: 27 / 50
Progress: 28 / 50
Progress: 29 / 50
Progress: 30 / 50
Progress: 31 / 50
Progress: 32 / 50
Progress: 33 / 50
Progress: 34 / 50
Progress: 35 / 50
Progress: 36 / 50
Progress: 37 / 50
Progress: 38 / 50
Progress: 39 / 50
Progress: 40 / 50
Progress: 41 / 50
Progress: 42 / 50
Progress: 43 / 50
Progress: 44 / 50
Progress: 45 / 50
Progress: 46 / 50
Progress: 47 / 50
Progress: 48 / 50
Progress: 49 / 50
Progress: 50 / 50
Vectors created.


# Updating the query vector as per Rocchio Algorithm
In this section, we update the expanded query vector for each query with each document using the Rocchio Algorithm. This algorithm uses the expanded query vector and already known information about relevance of query with each document which can be either 0 or 1.

In [ ]:
new_quer = []
a = 1
b = 0.75
c = 0.15

doc_names = []
for name in files : 
  doc_names.append(name[0].split('/')[3])

print("Creating new query vector using Rocchio Algorithm...")
for i in range(len(exp_quer)) : 
  count1 = 0.0
  count2 = 0.0
  demo1 = np.zeros((vect_size,),dtype=float)
  demo2 = np.zeros((vect_size,),dtype=float)
  queryNo = exp_quer[i][0]

  for k in range(len(refDoc)) :
    if(refDoc[k][0] == queryNo) : 
      index = doc_names.index(refDoc[k][2])
      if refDoc[k][3] == 1 : 
        demo1 = demo1 + doc_arr[index]
        count1 += 1
      if refDoc[k][3] == 0 : 
        count2 += 1
        demo2 = demo2 + doc_arr[index]

      if count1 == 0:
        count1 = 1
      if count2 == 0:
        count2 = 1
  temp_vect = np.zeros((vect_size,),dtype=float)
  temp_vect = temp_vect + np.multiply(q_arr[i], a) + np.multiply(demo1, b/count1) - np.multiply(demo2, c/count2)
  new_quer.append(temp_vect)

q_arr = new_quer
del new_quer
print("New Query vector created using Rocchio Algorithm.")

Creating new query vector using Rocchio Algorithm...
New Query vector created using Rocchio Algorithm.


# Calculating the cosine similarity
In this section, the cosine similarity is calculated between the new expanded query vector and the original document vector. And then rankings are calculated by sorting the similarity score.

In [ ]:
print("Calculating Rankings of Docs for each query...")

r1 = []

for i in range(len(q_arr)) :
    rTemp = []
    for j in range(len(doc_arr)) :
        cos_sim = dot(q_arr[i], doc_arr[j])/(norm(q_arr[i])*norm(doc_arr[j]))
        rTemp.append([files[j][0], cos_sim])
        
    rTemp.sort(key = lambda rTemp: rTemp[1], reverse=True) 
    r1.append(rTemp)

    print("Progress: ", i+1, " / ", len(q_arr))

print("Ranking Complete.")

Calculating Rankings of Docs for each query...
Progress:  1  /  50
Progress:  2  /  50
Progress:  3  /  50
Progress:  4  /  50
Progress:  5  /  50
Progress:  6  /  50
Progress:  7  /  50
Progress:  8  /  50
Progress:  9  /  50
Progress:  10  /  50
Progress:  11  /  50
Progress:  12  /  50
Progress:  13  /  50
Progress:  14  /  50
Progress:  15  /  50
Progress:  16  /  50
Progress:  17  /  50
Progress:  18  /  50
Progress:  19  /  50
Progress:  20  /  50
Progress:  21  /  50
Progress:  22  /  50
Progress:  23  /  50
Progress:  24  /  50
Progress:  25  /  50
Progress:  26  /  50
Progress:  27  /  50
Progress:  28  /  50
Progress:  29  /  50
Progress:  30  /  50
Progress:  31  /  50
Progress:  32  /  50
Progress:  33  /  50
Progress:  34  /  50
Progress:  35  /  50
Progress:  36  /  50
Progress:  37  /  50
Progress:  38  /  50
Progress:  39  /  50
Progress:  40  /  50
Progress:  41  /  50
Progress:  42  /  50
Progress:  43  /  50
Progress:  44  /  50
Progress:  45  /  50
Progress:  46  / 

# Calculating the MAP score using top 10 documents for each query

In [ ]:
#List containing AP10 for each query
AP10 = []

for i in range(len(r1)):
    cAP = 0
    tAP = []
    for j in range(min(10, len(r1[i]))):
        for l in range(j):
            docName = r1[i][j][0].split('/')[3]
            queryNo = queries[i][0]

            #Search for this doc in reference file
            for k in range(len(refDoc)):
                if(refDoc[k][0] == queryNo and refDoc[k][2] == docName and refDoc[k][3] == '1'):
                    cAP = cAP + 1
        tAP.append(cAP/(j+1))
    avgP = 0
    for val in tAP:
        avgP += val
    AP10.append(avgP/10)

MAP10 = 0
for ap in AP10:
    MAP10 += ap
MAP10 /= 50

In [ ]:
print('MAP using CBOW for new query vector using Rocchio Algorithm :', round(MAP10,2))

MAP using CBOW for new query vector using Rocchio Algorithm : 0.21
